# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 22 2022 10:40AM,247035,20,ATR-I000006,"HVL, LLC dba Atrium Innovations",Released
1,Sep 22 2022 10:40AM,247035,20,ATR-I000007,"HVL, LLC dba Atrium Innovations",Released
2,Sep 22 2022 10:39AM,247041,12,SB8240345,SugarBear,Released
3,Sep 22 2022 10:34AM,247040,12,SB8240344,SugarBear,Released
4,Sep 22 2022 10:32AM,247039,19,MSP214164,"Methapharm, Inc.",Released
5,Sep 22 2022 10:32AM,247039,19,MSP214149,"Methapharm, Inc.",Released
6,Sep 22 2022 10:32AM,247039,19,MSP214173,"Methapharm, Inc.",Released
7,Sep 22 2022 10:32AM,247039,19,MSP214175,"Methapharm, Inc.",Released
8,Sep 22 2022 10:32AM,247039,19,MSP214185,"Methapharm, Inc.",Released
9,Sep 22 2022 10:32AM,247039,19,MSP214227,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,247035,Released,2
22,247036,Released,1
23,247039,Released,7
24,247040,Released,1
25,247041,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247035,NaN,NaN,2.0
247036,NaN,NaN,1.0
247039,NaN,NaN,7.0
247040,NaN,NaN,1.0
247041,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246888,0.0,1.0,0.0
246889,0.0,1.0,0.0
246890,0.0,0.0,1.0
246928,0.0,0.0,2.0
246933,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246888,0,1,0
246889,0,1,0
246890,0,0,1
246928,0,0,2
246933,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246888,0,1,0
1,246889,0,1,0
2,246890,0,0,1
3,246928,0,0,2
4,246933,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246888,,1,
1,246889,,1,
2,246890,,,1
3,246928,,,2
4,246933,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations"
2,Sep 22 2022 10:39AM,247041,12,SugarBear
3,Sep 22 2022 10:34AM,247040,12,SugarBear
4,Sep 22 2022 10:32AM,247039,19,"Methapharm, Inc."
11,Sep 22 2022 10:17AM,247034,15,"Chadwick Pharmaceuticals, Inc."
12,Sep 22 2022 10:17AM,247036,10,Bio-PRF
13,Sep 22 2022 10:04AM,247033,10,Bio-PRF
21,Sep 22 2022 10:02AM,247031,20,"ACI Healthcare USA, Inc."
22,Sep 22 2022 9:53AM,247028,10,"Nextsource Biotechnology, LLC"
23,Sep 22 2022 9:50AM,247026,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations",,,2
1,Sep 22 2022 10:39AM,247041,12,SugarBear,,,1
2,Sep 22 2022 10:34AM,247040,12,SugarBear,,,1
3,Sep 22 2022 10:32AM,247039,19,"Methapharm, Inc.",,,7
4,Sep 22 2022 10:17AM,247034,15,"Chadwick Pharmaceuticals, Inc.",,,1
5,Sep 22 2022 10:17AM,247036,10,Bio-PRF,,,1
6,Sep 22 2022 10:04AM,247033,10,Bio-PRF,,,8
7,Sep 22 2022 10:02AM,247031,20,"ACI Healthcare USA, Inc.",,,1
8,Sep 22 2022 9:53AM,247028,10,"Nextsource Biotechnology, LLC",,,1
9,Sep 22 2022 9:50AM,247026,15,"Alliance Pharma, Inc.",,,41


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations",2,,
1,Sep 22 2022 10:39AM,247041,12,SugarBear,1,,
2,Sep 22 2022 10:34AM,247040,12,SugarBear,1,,
3,Sep 22 2022 10:32AM,247039,19,"Methapharm, Inc.",7,,
4,Sep 22 2022 10:17AM,247034,15,"Chadwick Pharmaceuticals, Inc.",1,,
5,Sep 22 2022 10:17AM,247036,10,Bio-PRF,1,,
6,Sep 22 2022 10:04AM,247033,10,Bio-PRF,8,,
7,Sep 22 2022 10:02AM,247031,20,"ACI Healthcare USA, Inc.",1,,
8,Sep 22 2022 9:53AM,247028,10,"Nextsource Biotechnology, LLC",1,,
9,Sep 22 2022 9:50AM,247026,15,"Alliance Pharma, Inc.",41,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations",2,,
1,Sep 22 2022 10:39AM,247041,12,SugarBear,1,,
2,Sep 22 2022 10:34AM,247040,12,SugarBear,1,,
3,Sep 22 2022 10:32AM,247039,19,"Methapharm, Inc.",7,,
4,Sep 22 2022 10:17AM,247034,15,"Chadwick Pharmaceuticals, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations",2.0,NaN,NaN
1,Sep 22 2022 10:39AM,247041,12,SugarBear,1.0,NaN,NaN
2,Sep 22 2022 10:34AM,247040,12,SugarBear,1.0,NaN,NaN
3,Sep 22 2022 10:32AM,247039,19,"Methapharm, Inc.",7.0,NaN,NaN
4,Sep 22 2022 10:17AM,247034,15,"Chadwick Pharmaceuticals, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations",2.0,0.0,0.0
1,Sep 22 2022 10:39AM,247041,12,SugarBear,1.0,0.0,0.0
2,Sep 22 2022 10:34AM,247040,12,SugarBear,1.0,0.0,0.0
3,Sep 22 2022 10:32AM,247039,19,"Methapharm, Inc.",7.0,0.0,0.0
4,Sep 22 2022 10:17AM,247034,15,"Chadwick Pharmaceuticals, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2222995,39.0,5.0,1.0
12,494081,2.0,0.0,0.0
15,494060,42.0,0.0,0.0
19,740925,8.0,1.0,0.0
20,494066,3.0,0.0,0.0
21,1481597,3.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2222995,39.0,5.0,1.0
1,12,494081,2.0,0.0,0.0
2,15,494060,42.0,0.0,0.0
3,19,740925,8.0,1.0,0.0
4,20,494066,3.0,0.0,0.0
5,21,1481597,3.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,39.0,5.0,1.0
1,12,2.0,0.0,0.0
2,15,42.0,0.0,0.0
3,19,8.0,1.0,0.0
4,20,3.0,0.0,0.0
5,21,3.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,12,Released,2.0
2,15,Released,42.0
3,19,Released,8.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,5.0,0.0,0.0,1.0,0.0,3.0
Released,39.0,2.0,42.0,8.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,1.0,0.0,3.0
2,Released,39.0,2.0,42.0,8.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,1.0,0.0,3.0
2,Released,39.0,2.0,42.0,8.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()